# Market Feed Stability Test

This notebook demonstrates how to use the `FeedManager` to connect to **multiple data sources simultaneously** (Deribit and Bloomberg) and monitor their stability.

**Objective:** Run the feed for a set duration and continuously print the latest index prices from each source to verify that both are streaming data correctly.

In [1]:
# %pip install -e .
# %pip install pandas
# %pip install --index-url=https://blpapi.bloomberg.com/repository/releases/python/simple blpapi

In [2]:
import time
import json
import os
import sys
from datetime import datetime
from IPython.display import clear_output, display
import pandas as pd

# Add the local source folder to the path
sys.path.append(os.path.abspath("src"))
from market_feed import FeedManager

print("✅ Imports successful.")

✅ Imports successful.


### Step 1: Create a Multi-Source Configuration

We will define a configuration that tracks instruments from both `deribit` and `bloomberg`.

In [3]:
multi_source_config = [
    {
        "tab_name": "BTC_DERIBIT",
        "base_symbol": "BTC",
        "settlement": "coin",
        "source": "deribit" 
    },
    {
        "tab_name": "ESH1_BBG",
        "base_symbol": "SPY US Equity",
        "settlement": "usd",
        "source": "bloomberg"
    }
]

# Create a dummy config file for the manager
with open("multi_config.json", "w") as f:
    json.dump(multi_source_config, f)

print("📄 multi_config.json created successfully.")

📄 multi_config.json created successfully.


### Step 2: Initialize the FeedManager

We'll initialize the `FeedManager` using our new config. For this demo, we assume you have `keys.json` for Deribit access and that your Bloomberg terminal connection is running locally.

In [4]:
# The FeedManager will automatically detect the 'bloomberg' source
# and initialize the BloombergAdapter.
feed = FeedManager(config_path="multi_config.json", keys_path="keys.json")

print("✅ Feed Manager initialized.")
print("-> Detected Adapters:", list(feed.adapters.keys()))

# Check bootstrap prices
snap = feed.get_snapshot()
print("-> Bootstrap Prices:", snap.index_prices)

[FeedManager] Bloomberg Adapter Initialized.
[FeedManager] Bootstrapping...
[Bloomberg] Fetching chain for SPY US Equity...
[FeedManager] Bootstrapped BTC-PERPETUAL: 69612.2
[FeedManager] Bootstrapped BTC_USDC: 69612.41
[FeedManager] Bootstrapped SPY US Equity: 681.75
✅ Feed Manager initialized.
-> Detected Adapters: ['deribit', 'bloomberg']
-> Bootstrap Prices: {'BTC-PERPETUAL': 69612.2, 'BTC_USDC': 69612.41, 'SPY US Equity': 681.75}


### Step 3: Start the Stream and Monitor

We'll start the feed and run a loop for 60 seconds. In each iteration, we will:
1. Get the latest market snapshot.
2. Display the status and latest index prices in a clean table.
3. Clear the output to create a live-updating display.

In [5]:
feed.start_stream()
print("Starting streams... Waiting 5 seconds for connections to establish.")
time.sleep(5)
print("--- MONITORING STARTED ---")

run_duration_seconds = 60
start_time = time.time()

try:
    while time.time() - start_time < run_duration_seconds:
        clear_output(wait=True)
        
        snap = feed.get_snapshot()
        
        # Prepare data for display
        status = {
            'Feed Status': '✅ Ready' if snap.is_ready else '⚠️ Not Ready',
            'Timestamp': datetime.now().strftime('%H:%M:%S')
        }
        prices = snap.index_prices
        
        # Display as a DataFrame
        status_df = pd.DataFrame([status])
        prices_df = pd.DataFrame(prices.items(), columns=['Symbol', 'Price'])
        prices_df.set_index('Symbol', inplace=True)
        
        print("--- Market Feed Stability Monitor ---")
        display(status_df)
        print("\nLatest Index Prices:")
        if not prices_df.empty:
            display(prices_df)
        else:
            print("Waiting for price updates...")
        
        # Check if we should stop
        elapsed = time.time() - start_time
        print(f"\nRunning for {int(elapsed)} / {run_duration_seconds} seconds...")
        
        time.sleep(2) # Update every 2 seconds

except KeyboardInterrupt:
    print("\nMonitoring interrupted by user.")
finally:
    clear_output(wait=True)
    print("--- MONITORING ENDED ---")
    # Always ensure the stream is stopped
    feed.stop_stream()
    print("✅ Feed streams stopped successfully.")


--- MONITORING ENDED ---
✅ Feed streams stopped successfully.
